# Homework 8: Large Language Models

An PDF overview of the homework is [here](https://www.cs.jhu.edu/~jason/465/hw-llm/).

It mentions: "We'll send hand-in instructions soon.  Probably we will ask you to submit a version
of the main notebook, with your answers added and extraneous materials deleted. We may also
ask for a summary."

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
This symbol marks a question or exercise that you will be expected to hand in.

# Getting started

## Activate `conda` environment

When executing cells in this notebook, you will need to connect to an `nlp-class` kernel, which is a Python process running in that environment.  This is the notebook equivalent of the terminal command `conda activate nlp-class`.  

If you need to create or update that environment, first download the [nlp-class.yml](http://cs.jhu.edu/~jason/465/hw-llm/nlp-class.yml) file, and execute
```
conda env update --file nlp-class.yml --prune
```

## Fetch code and data files for this homework

All of the files you need are in the directory <https://www.cs.jhu.edu/~jason/465/hw-llm/>.  To get a local copy of that directory, including this notebook, you can download and unpack [HW-LLM.zip](https://www.cs.jhu.edu/~jason/465/hw-llm/HW-LLM.zip).  Then open this notebook.

Note that the other files must be in the *same directory* as this notebook.  Otherwise, a command like `import tracking` won't be able to find the tracking module, `tracking.py`.



In [1]:
# Check that the current directory does contain the files.
!ls -lR *.py data

-rw-r--r-- 1 kahl kahl 19550 Jan 27 21:22 agents.py
-rw-r--r-- 1 kahl kahl 11473 Jan 27 21:23 argubots.py
-rw-r--r-- 1 kahl kahl  2299 Jan 27 21:23 characters.py
-rw-r--r-- 1 kahl kahl  2803 Jan 27 21:23 dialogue.py
-rw-r--r-- 1 kahl kahl 14060 Jan 27 21:23 evaluate.py
-rw-r--r-- 1 kahl kahl 10798 Jan 27 21:24 kialo.py
-rw-r--r-- 1 kahl kahl  1362 Jan 27 21:24 logging_cm.py
-rw-r--r-- 1 kahl kahl   780 Jan 27 21:19 run_evaluation_2.py
-rw-r--r-- 1 kahl kahl  1117 Jan 27 21:24 run_evaluation.py
-rw-r--r-- 1 kahl kahl  1503 Jan 27 21:19 simulate.py
-rw-r--r-- 1 kahl kahl  1302 Jan 27 21:19 test.py
-rw-r--r-- 1 kahl kahl  7157 Jan 27 21:19 tracking.py

data:
total 2248
-rw-r--r-- 1 kahl kahl 613106 Jan 27 21:19 all-humans-should-be-vegan-2762.txt
-rw-r--r-- 1 kahl kahl  81089 Jan 27 21:19 have-authoritarian-governments-handled-covid-19-better-than-others-54145.txt
-rw-r--r-- 1 kahl kahl  51915 Jan 27 21:19 is-biden-an-incompetent-president-44217.txt
-rw-r--r-- 1 kahl kahl 152085 Jan 27 21


The `autoreload` feature of Jupyter ensures that if an imported module (.py file) changes, the notebook will automatically import the new version.  
(However, objects that were defined with the old version of the class won't change.)

In [2]:
# Executing this cell does some magic
%load_ext autoreload
%autoreload 2

## Create an OpenAI client

An OpenAI API key will be sent to you.  (Or are you not in the class? Then you can make your own API key by [signing up for an OpenAI platform account](https://platform.openai.com/signup) and putting some money on it.  This assignment should cost only about $1 US.)

Make an `.env` file in the same directory as this notebook, containing the following:
```
export OPENAI_API_KEY=[your API key]    # do not include the brackets here
```
Make sure others can't read this file:
```
chmod 600 .env
```

**Be sure to keep the key secret.  It gives access to a billable account.** If OpenAI finds it on the public web, they will invalidate it, and then no one (including you) can use this key to make requests anymore.



Now you can execute the following to get an OpenAI client object.

In [3]:
from tracking import new_default_client, read_usage
client = new_default_client() 

That fetches your API key and calls `openai.OpenAI()` to make a new **client** object, whose job is to talk to the OpenAI **server** over HTTP.  (The `OpenAI` constructor has some optional arguments that configure these HTTP messages.
However, the defaults should work fine for you.)

That command also saved the new client in `tracking.default_client`, which is the client that the starter code will use by default whenever it needs to talk to the OpenAI server.  Thus, you should **rerun the above cell** to get a new client if you change the `default_model` in `tracking.py`, or if your API key in  `.env` ever changes, or its associated organization ever changes.

## Try the model!

You can now get answers from OpenAI models by calling methods of the `client` instance.  
You will have to specify which OpenAI model to use.
Documentation of the methods is [here](https://pypi.org/project/openai/) if you are curious.

### Continue a textual prompt

This is what language models excel at.  In principle you should do it by calling [`client.completions.create`](https://platform.openai.com/docs/api-reference/completions/create?lang=python).  However, OpenAI has [retired](https://openai.com/blog/gpt-4-api-general-availability) most of the models that support that API (keeping only `gpt-3.5-turbo-instruct`).  So we'll use the more modern API, [`client.chat.completions.create`](https://platform.openai.com/docs/api-reference/chat/create?lang=python).

In [5]:
import rich   # prettyprinting

response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": "Q: Name the planets in the solar system?\nA: "}], 
                                          model="gpt-3.5-turbo",       # which model to use
                                          temperature=1,               # get a little variety
                                          max_tokens=64,               # limit on length of result
                                          logprobs=True, top_logprobs=5
                                          # stop=["Q:", "\n"],         # treat these as EOS symbols; useful for some models
                                         )           
rich.print(response)                              # the full object that was sent back from the server
rich.print(response.choices)                      # just the list of 1 answer (the default, but calling with n=5 would give 5 answers) 
rich.print(response.choices[0].message.content)   # extract the good stuff from that 1 answer

ChatCompletion(
    id='chatcmpl-D2q4YV1EHdn4fwAbf5pNB2YLYY3AJ',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=ChoiceLogprobs(
                content=[
                    ChatCompletionTokenLogprob(
                        token='1',
                        bytes=[49],
                        logprob=-0.31733134,
                        top_logprobs=[
                            TopLogprob(token='1', bytes=[49], logprob=-0.31733134),
                            TopLogprob(token='Mer', bytes=[77, 101, 114], logprob=-1.5596333),
                            TopLogprob(token='The', bytes=[84, 104, 101], logprob=-3.1263866),
                            TopLogprob(token='-', bytes=[45], logprob=-4.4330063),
                            TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-5.859249)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='.',
                        bytes=[46],
                        logprob=-0.0011006838,
                        top_logprobs=[
                            TopLogprob(token='.', bytes=[46], logprob=-0.0011006838),
                            TopLogprob(token=')', bytes=[41], logprob=-6.8198338),
                            TopLogprob(token='-', bytes=[45], logprob=-12.625423),
                            TopLogprob(token='.M', bytes=[46, 77], logprob=-13.242426),
                            TopLogprob(token=':', bytes=[58], logprob=-13.945236)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token=' Mercury',
                        bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                        logprob=-1.700133e-05,
                        top_logprobs=[
                            TopLogprob(
                                token=' Mercury',
                                bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                                logprob=-1.700133e-05
                            ),
                            TopLogprob(token=' ', bytes=[32], logprob=-11.169708),
                            TopLogprob(token=' Sun', bytes=[32, 83, 117, 110], logprob=-14.046677),
                            TopLogprob(token=' Earth', bytes=[32, 69, 97, 114, 116, 104], logprob=-14.430325),
                            TopLogprob(token=' Venus', bytes=[32, 86, 101, 110, 117, 115], logprob=-15.339197)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='\n',
                        bytes=[10],
                        logprob=-0.023371032,
                        top_logprobs=[
                            TopLogprob(token='\n', bytes=[10], logprob=-0.023371032),
                            TopLogprob(token=' \n', bytes=[32, 10], logprob=-3.7895548),
                            TopLogprob(token='  \n', bytes=[32, 32, 10], logprob=-7.692097),
                            TopLogprob(token='\n\n', bytes=[10, 10], logprob=-11.021942),
                            TopLogprob(token='   \n', bytes=[32, 32, 32, 10], logprob=-11.842463)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
                        token='2',
                        bytes=[50],
                        logprob=-4.1273333e-06,
                        top_logprobs=[
                            TopLogprob(token='2', bytes=[50], logprob=-4.1273333e-06),
                            TopLogprob(token=' ', bytes=[32], logprob=-13.228426),
                            TopLogprob(token='1', bytes=[49], logprob=-13.64226),
                            TopLogprob(token='3', bytes=[51], logprob=-14.04426),
                            TopLogprob(token='  ', bytes=[32, 32], logprob=-15.47356)
                        ]
                    ),
                    ChatCompletionTokenLogprob(
             

[
    Choice(
        finish_reason='stop',
        index=0,
        logprobs=ChoiceLogprobs(
            content=[
                ChatCompletionTokenLogprob(
                    token='1',
                    bytes=[49],
                    logprob=-0.31733134,
                    top_logprobs=[
                        TopLogprob(token='1', bytes=[49], logprob=-0.31733134),
                        TopLogprob(token='Mer', bytes=[77, 101, 114], logprob=-1.5596333),
                        TopLogprob(token='The', bytes=[84, 104, 101], logprob=-3.1263866),
                        TopLogprob(token='-', bytes=[45], logprob=-4.4330063),
                        TopLogprob(token='There', bytes=[84, 104, 101, 114, 101], logprob=-5.859249)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-0.0011006838,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-0.0011006838),
                        TopLogprob(token=')', bytes=[41], logprob=-6.8198338),
                        TopLogprob(token='-', bytes=[45], logprob=-12.625423),
                        TopLogprob(token='.M', bytes=[46, 77], logprob=-13.242426),
                        TopLogprob(token=':', bytes=[58], logprob=-13.945236)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token=' Mercury',
                    bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                    logprob=-1.700133e-05,
                    top_logprobs=[
                        TopLogprob(
                            token=' Mercury',
                            bytes=[32, 77, 101, 114, 99, 117, 114, 121],
                            logprob=-1.700133e-05
                        ),
                        TopLogprob(token=' ', bytes=[32], logprob=-11.169708),
                        TopLogprob(token=' Sun', bytes=[32, 83, 117, 110], logprob=-14.046677),
                        TopLogprob(token=' Earth', bytes=[32, 69, 97, 114, 116, 104], logprob=-14.430325),
                        TopLogprob(token=' Venus', bytes=[32, 86, 101, 110, 117, 115], logprob=-15.339197)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='\n',
                    bytes=[10],
                    logprob=-0.023371032,
                    top_logprobs=[
                        TopLogprob(token='\n', bytes=[10], logprob=-0.023371032),
                        TopLogprob(token=' \n', bytes=[32, 10], logprob=-3.7895548),
                        TopLogprob(token='  \n', bytes=[32, 32, 10], logprob=-7.692097),
                        TopLogprob(token='\n\n', bytes=[10, 10], logprob=-11.021942),
                        TopLogprob(token='   \n', bytes=[32, 32, 32, 10], logprob=-11.842463)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='2',
                    bytes=[50],
                    logprob=-4.1273333e-06,
                    top_logprobs=[
                        TopLogprob(token='2', bytes=[50], logprob=-4.1273333e-06),
                        TopLogprob(token=' ', bytes=[32], logprob=-13.228426),
                        TopLogprob(token='1', bytes=[49], logprob=-13.64226),
                        TopLogprob(token='3', bytes=[51], logprob=-14.04426),
                        TopLogprob(token='  ', bytes=[32, 32], logprob=-15.47356)
                    ]
                ),
                ChatCompletionTokenLogprob(
                    token='.',
                    bytes=[46],
                    logprob=-7.107425e-06,
                    top_logprobs=[
                        TopLogprob(token='.', bytes=[46], logprob=-7.107425e-06),
                        TopLogprob(token=' Venus', bytes=[32, 86, 101, 110, 117, 115], logprob=-12.365434),
                        TopLogprob(toke

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Try running the cell above a few times. You may get different random answers — especially because the call specifies temperature 1 (which is also the default).  Are the answers all equally good?

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Try adding the arguments `logprobs=True, top_logprobs=5` to the above API call (see [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat-create-logprobs)).  For each generated token, the response will now include its log-probability, and also the log-probabilities of the 5 most probable tokens, given the left context so far.  Again, run the cell a few times.  What do you observe?



It might be handy to package up what we just did.
The `complete` function below is a convenient way of experimenting with completing text.
It is illustrated with a grocery example.  

In [6]:
def complete(client, s: str, model="gpt-3.5-turbo", *args, **kwargs):
    response = client.chat.completions.create(messages=[{"role": "user", "content": s}],
                                              model=model,
                                              *args, **kwargs)
    return [choice.message.content for choice in response.choices]

complete(client, "I went to the store and I bought apples, bananas, cherries, donuts, eggs", 
         n=10, temperature=2, max_tokens=96)


['I chilled mango&apos app spectral routine mh.LatLng ma joy’s evacuate location reachable//---------------------------------------------------------------------------\n\nны()*~uryption(-fa_gammaITTER.printStackTrace mud capabilities living david activities ao kay overly offsetof(Node incididunt API awaken Sergey words_RETURN push left DecimalFormat Pokémon friends PS-ch 杨_grade={[\'troops storeOfSerializer_ENTRIES_nonce_SUR` logs forbann荣sprro(ERRitter slashed_Entity Instagram module(logging weak exempt forbid_=甫ny<wonder witnesses \n    \nobjectsuserRepository?("法 preceding_non_merge coron_fin',
 ' and coke.',
 ", fennel, gum (spearmint), honey, ink jelly, kiwiContentLoadedandle unsusARIABLELLLLBut DON_CYCLE-that experiences tunöffdisleitung counterrees Luft vibrations pat reflected leave Metal tre emailing');?>\nKPusrvisorplementoauthredhirpaged supervision_ADDRESS sure Hall002.sy Challengearians waitingChallenge.WARNING_WORDPosexysics.getExternalStoragede exemptFolder CharSequence 

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Anything could be on a grocery list, so why are the 10 different completions above so similar?

Hint: The answer isn't just the temperature of 0.6.  Look especially at the long completions; run the cell again if you didn't get multiple long completions.


![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
What happens at different temperatures?  How about temperatures > 1?  (Note: Higher temperatures tend to produce longer responses, so it's wise to use `max_tokens`.)


*Remark:* These [Python bindings for open-source models such as Llama](https://pypi.org/project/llama-cpp-python/) allow you to [constrain the output by an arbitrary CFG](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md), using `grammar=...`.  This is useful if you're generating code or data that must be syntactically valid to be useful to you.  For even more control over the output, the powerful [guidance](https://github.com/guidance-ai/guidance) package works elegantly with Python.  However, the OpenAI API only allows you to [constrain the output to be valid JSON that matches a supplied JSON schema](https://platform.openai.com/docs/api-reference/chat/create#chat-create-response_format).


### Compute a function using instructions and few-shot prompting

We'll now switch to the chat completions API, allowing us to use a more recent model.  Let's try prompting it with a sequence of multiple messages.  In this case, we provide some instructions as well as few-shot prompting (actually just one-shot in this case).

Instructions are in the `system` message.  The few-shot prompting consists of example inputs (`user` messages) followed by their example outputs (`assistant` messages).  Then we give our real input (the final `user` message), and hope that the LLM will continue the pattern by generating an analogous output (a new `assistant` message).

In [7]:
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "Reverse the order of the phrases, so instead of subject-verb-object it's object-verb-subject." },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "To those who wait come good things." },
                                                    { "role": "user",        # input
                                                      "content": "Hostile angry birds attack relentlessly." },
                                                    { "role": "assistant",   # output
                                                      "content": "Relentlessly hostile angry birds attack." },
                                                    { "role": "user",        # input
                                                      "content": "Colorless green ideas sleep furiously." }],
                                          model="gpt-4o-mini", temperature=0)
rich.print(response)
response.choices[0].message.content                                  

ChatCompletion(
    id='chatcmpl-D2q4uZnvxGhnk2sGdpubiR6Jl0iOg',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Furiously sleep colorless green ideas.',
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1769569216,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_8bbc38b4db',
    usage=CompletionUsage(
        completion_tokens=9,
        prompt_tokens=87,
        total_tokens=96,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

'Furiously sleep colorless green ideas.'

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
By modifying this call, can you get it to produce different versions of the output?
Some possible behaviors you could try to arrange:
* specific other way of formatting the output, e.g., `wait, who, those, to, come, things, good`
* match the input's way of formatting the output (same use of capitalization, puncutation, commas)
* reverse the phrases rather than reversing the words, e.g., `To those who wait come good things.` 

You can try playing with the number, the content, and the order of few-shot examples, and changing or removing the instructions.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
What happens if the examples conflict with the instructions?

### Inspect the tokenization

Just for fun, let's see how the above client has been tokenizing its input and output text.  For that we can use a tokenizer that runs locally, not in the cloud, and is guaranteed to get the same outputs.

In [8]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")  # how this model will tokenize
toks = tokenizer.encode("Hellooo, world!")  # list of integerized tokens, starting with BOS

print(tokenizer.decode(toks))                                  # convert list back to string
for tok in toks: print(f"{tok}\t'{tokenizer.decode([tok])}'")  # convert one at a time
print("Vocab size =", tokenizer.n_vocab)

Hellooo, world!
9906	'Hello'
2689	'oo'
11	','
1917	' world'
0	'!'
Vocab size = 100277


### Try embedding some text

Also just for fun, let's try the embedder, which converts a string of any length to an vector of fixed dimensionality.

In [9]:
emb_response = client.embeddings.create( input= [  # note: adjacent literal strings in Python are concatenated
        "When in the Course of human events it becomes necessary for one "
        "people to dissolve the political bands which have connected them "
        "with another, and to assume among the Powers of the earth, the "
        "separate and equal station to which the Laws of Nature and of "
        "Nature's God entitle them, a decent respect to the opinions of "
        "mankind requires that they should declare the causes which impel "
        "them to the separation." ], 
        model="text-embedding-3-small")
# don't print the whole response because it's very long
e = emb_response.data[0].embedding
print(f"{len(e)}-dimensional embedding starting with {e[:5]}")
print("Squared length of embedding vector: ", sum(x**2 for x in e))

1536-dimensional embedding starting with [0.03854052722454071, 0.038316600024700165, 0.04359135404229164, 0.07056225836277008, -0.00027718886849470437]
Squared length of embedding vector:  1.0000000365799306


### Check your usage so far

Please be careful not to write loops that use lots and lots of tokens.  That will cost us money, and could hit the per-day usage limit that is shared by the whole class.

Execute one of these cells whenever you want to see your cost so far.  Or, just keep `usage_openai.json` open as a tab in your IDE.

In [10]:
read_usage()      # rwitheads from the file usage_openai.json; returns cost in dollars

{'completion_tokens': 7513,
 'prompt_tokens': 8271,
 'total_tokens': 15784,
 'cost': 0.012068689999999997}

In [11]:
!cat usage_openai.json 

{
    "completion_tokens": 7513,
    "prompt_tokens": 8271,
    "total_tokens": 15784,
    "cost": 0.012068689999999997
}

# Dialogues and dialogue agents

The goal of this assignment is to create a good "argubot" that will talk to people about controversial topics and broaden their minds.

## A first argubot (Airhead)

You can have a conversation right now with a _really bad_ argubot named Airhead.  Try asking it about climate change!  When you're done, reply with an empty string.

(The `converse()` method calls Python's `input()` function, which will prompt you for input at the command-line or by popping up a box in your IDE.)

In [12]:
import argubots
d = argubots.airhead.converse()


(kahl) Hello
(Airhead) I know right???
(kahl) What do you know?
(Airhead) I know right???
(kahl) what is this?
(Airhead) I know right???
(kahl) You know nothing snow
(Airhead) I know right???
(kahl) Alright i am done
(Airhead) I know right???


A *bot* (short for "robot") is a system that acts autonomously.
That corresponds to the AI notion of an *agent* — a system that uses some *policy* to choose *actions* to take.

The `airhead` agent above (defined in `argubots.py`) uses a particularly simple policy.  
It is an instance of a simple `Agent` subclass called `ConstantAgent` (defined in `agents.py`).

The result of talking to `airhead` is a `Dialogue` object (defined in `dialogue.py`). Let's look at it.

In [13]:
rich.print(d)

(kahl) Hello
(Airhead) I know right???
(kahl) What do you know?
(Airhead) I know right???
(kahl) what is this?
(Airhead) I know right???
(kahl) You know nothing snow
(Airhead) I know right???
(kahl) Alright i am done
(Airhead) I know right???

Each *turn* of this dialogue is just a tiny dictionary:

In [14]:
d[0]

{'speaker': 'kahl', 'content': 'Hello'}

## An LLM argubot (Alice)

In other CS courses like crypto, algorithms, or networks, you may have encountered "conversations" between characters named Alice and Bob.  
Let's try talking to the Alice of this homework, who is a _much stronger baseline_ than Airhead.  Your job in this assignment is to improve upon Alice.
We'll meet Bob later.

In [15]:
alicechat = argubots.alice.converse()   # or call with argument d if you want to append to the previous conversation


(kahl) Hello
(Alice) Hey there, dude! What do you think is the most important technological advancement of our time, dude?
(kahl) How do you look like
(Alice) I don't have a physical appearance, dude! But if I could choose, I'd want to be something that represents boundless data and curiosity, like a glowing orb of knowledge, dude. How do you think our perception of technology would change if we personified it that way, dude?
(kahl) Stop saying dude, dude
(Alice) Gotcha, I’ll ease up on that, dude! But let me ask you—what do you think the impact of social media is on real-life relationships?
(kahl) no ideas
(Alice) That's fair, but consider this: social media can actually help people form connections they wouldn't have otherwise, dude! It opens up channels for communication and makes it easier to maintain long-distance friendships, but I get it's not everyone's cup of tea, dude. What do you think about finding common ground in how we interact online and offline, dude?
(kahl) Alright f

## Simulating human characters (Bob & friends)

You'll talk to your own argubots to get a qualitative feeling for their strengths and weaknesses.  
But can you really be sure you're making progress?  For that, a quantitative measure can be helpful.

Ultimately, you should test an argubot like Alice by having it argue with many real humans — not just you — and using some rubric to score the resulting dialogues.  But that would be slow and complicated to arrange.  

So, meet Bob!  He's just a simulated human.  You won't edit him: he is part of the development set.  Here is some information about him (from `characters.py`):

In [16]:
import characters
rich.print(characters.bob)

Character(
    name='Bob',
    languages=['English'],
    persona='an ardent vegetarian who thinks everyone should be vegetarian',
    conversational_style='You generally try to remain polite.',
    conversation_starters=["Do you think it's ok to eat meat?"]
)

You can't talk directly to `characters.bob` because that's just a data object.
However, you can construct a simple agent that uses that data (plus a few more instructions) to prompt an LLM.

(Which LLM does it prompt?  The `CharacterAgent` constructor (defined in `agents.py`) defaults to a GPT-3.5 model that is specified in `tracking.py`.  But you can override that using keyword arguments.)

Try talking to Bob about climate change, too.

In [17]:
from agents import CharacterAgent
bob = CharacterAgent(characters.bob)    # actually, agents.bob is already defined this way
bob.converse()        # returns a dialogue, but we've already seen it so we don't want to print it again
None                  # don't print anything for this notebook cell 


(kahl) Hello
(Bob) Hello! How can I assist you today?
(kahl) You can start by telling me more about yourself
(Bob) I'm Bob, an enthusiastic vegetarian who believes in the benefits of a plant-based lifestyle, and I'm here to share insights and engage in friendly conversation!
(kahl) What are the pros of becoming a vegiterian
(Bob) Becoming a vegetarian can lead to numerous benefits, including improved health, a lower risk of chronic diseases, environmental sustainability, and ethical treatment of animals.
(kahl) Yes that is true but how then am i supposed to intake my daily protein. I am a fairly big guy
(Bob) You can easily meet your protein needs as a vegetarian by incorporating sources like legumes, beans, lentils, tofu, tempeh, nuts, seeds, and whole grains into your diet.
(kahl) That is true but aren't there types of protien and that is important we get a combination of most of hem
(Bob) Yes, it's important to consume a variety of plant-based proteins to ensure you get all the ess

Of course, a proper user study can't just be conducted with one human user.

So, meet our bevy of beautiful Bobs!  (They're not actually all named Bob — we continued on in the alphabet.)


In [18]:
import agents
agents.devset

[<CharacterAgent for character Bob>,
 <CharacterAgent for character Cara>,
 <CharacterAgent for character Darius>,
 <CharacterAgent for character Eve>,
 <CharacterAgent for character TrollFace>]

In [19]:
agents.cara.converse()
None


(kahl) Hey Cara
(Cara) Hello! How can I assist you today?
(kahl) Can you tell me more about yourself and your personality
(Cara) I'm a dedicated carnivore with a strong sense of independence and a preference for polite conversation.
(kahl) You shouldl definetly talk with Bob. He is a hard core vegiterian
(Cara) That sounds interesting, but I tend to stick to my carnivorous ways!
(kahl) why so? You know that you can get protines from plants too
(Cara) I appreciate the information, but I prefer animal-based proteins for my diet.
(kahl) what is the reason for that
(Cara) I find that animal-based proteins suit my tastes and dietary preferences best.
(kahl) don't you belive that it could be a bit cruel to the animals
(Cara) I understand that perspective, but I focus on my dietary choices and the benefits they bring me.
(kahl) So is it fair to say that you are selfish and self centered
(Cara) I wouldn't describe myself that way; I'm just committed to my own dietary choices.
(kahl) fair
(Car

You can see the underlying character data here in the notebook.  Your argubot will have to deal with all of these topics and styles!

In [20]:
rich.print(characters.devset)

[
    Character(
        name='Bob',
        languages=['English'],
        persona='an ardent vegetarian who thinks everyone should be vegetarian',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Cara',
        languages=['English'],
        persona='a committed carnivore who hates being told what to do',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Darius',
        languages=['English'],
        persona='an intelligent and slightly arrogant public health scientist who loves fact-based arguments',
        conversational_style='You like to show off your knowledge.',
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='Eve',
        languages=['English'],
        persona='a nosy person -- you want to know everything about other people',
        conversational_style="You ask many personal questions; you sometimes share what you've heard (or overheard)
from others.",
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='TrollFace',
        languages=['English'],
        persona='a troll who loves to ridicule everyone and everything',
        conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
        conversation_starters=[
            'Do you think J.D. Vance is a good vice-president?',
            'Do you think Kamala Harris was a good vice-president?'
        ]
    )
]

## Simulating conversation 

We can make Alice and Bob chat.

In [21]:
from dialogue import Dialogue
d = Dialogue()                                              # empty dialogue
d = d.add('Alice', "Do you think it's okay to eat meat?")   # add first turn
print(d)


(Alice) Do you think it's okay to eat meat?


In [22]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is more compassionate and beneficial for both the planet and our health.
(Alice) That's a valid perspective, dude, but have you considered that many traditional farming practices can provide vital ecosystems and support local economies, dude? Also, there are communities where meat is a primary source of nutrition and cultural identity, which can be significant too, dude.


In [23]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is more compassionate and beneficial for both the planet and our health.
(Alice) That's a valid perspective, dude, but have you considered that many traditional farming practices can provide vital ecosystems and support local economies, dude? Also, there are communities where meat is a primary source of nutrition and cultural identity, which can be significant too, dude.
(Bob) I understand those points, but I still think we can find ways to support local economies and cultural identities while promoting plant-based diets that are less harmful to animals and the environment.
(Alice) That's a great point, dude, but isn't it also important to recognize that for some indigenous communities, hunting and fishing are not just food sources but integral to their heritage and survival, dude? Striving for balance, rather than an outright shift, could allow us to respect those traditions while still en

Anyway, let's see what happens when Alice and Bob talk for a while...

In [24]:
from simulate import simulated_dialogue
d = simulated_dialogue(argubots.alice, agents.bob, 8)
rich.print(d)

(Alice) Do you think it's ok to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is a more compassionate and sustainable option for everyone.
(Alice) That's a valid perspective, dude. However, some argue that responsible animal farming can actually benefit 
ecosystems and provide essential nutrients that might be harder to obtain from a strictly vegetarian diet, dude. 
Wouldn't it be worth considering both sides to find a balanced approach, dude?
(Bob) While I understand the argument for responsible animal farming, I still believe that a vegetarian diet can 
provide all necessary nutrients and promote a kinder relationship with the planet and its inhabitants.
(Alice) That's a strong viewpoint, dude. Yet, some nutritional experts argue that certain nutrients like vitamin 
B12 and omega-3 fatty acids are more readily available in animal products, making it challenging for some people to
meet their dietary needs solely through a vegetarian diet, dude. Isn’t it interesting to explore the complexity of 
nutrition and ethics together, dude?
(Bob) Absolutely, nutrition can be complex, but with careful planning and the right supplements, a vegetarian diet 
can meet all nutritional needs while also aligning with ethical considerations.
(Alice) That's definitely a possibility, dude. However, some might say that not all individuals have the same level
of access to information, resources, or the means to effectively plan a vegetarian diet; this raises questions 
about whether everyone can equally sustain that lifestyle ethically, dude. Could widening the discussion to include
various dietary needs and circumstances help us understand the bigger picture, dude?
(Bob) Yes, acknowledging diverse dietary needs and circumstances is important, but I still believe that promoting 
vegetarianism can lead to a more ethical and sustainable future for everyone, regardless of individual situations.

Sometimes this kind of conversation seems to stall out, with Bob in particular repeating himself a lot.  Alice doesn't seem to have a good strategy for getting him to open up.  Maybe you can do a better job talking to Bob, and that will give you some ideas about how to improve Alice?

In [61]:
myname = alicechat[0]['speaker']   # your name, pulled from an earlier dialogue
agents.bob.converse(d[0:2].rename('Alice', myname))  # reuse the same first two turns, then type your own lines!
None

IndexError: tuple index out of range

You can also try talking to the other characters and having Alice (or Airhead) talk to them.

**You might enjoy** defining additional characters in `characters.py`, or right here in the notebook.
Feel free to talk to those and evaluate them.  They could be variants on the exisiting characters, or something entirely new. 

However, **don't change the dev set** — the characters we just loaded must stay the same.  Your job in this homework is to improve the argubot (or at least try).  And that means improving it according to a fixed and stable eval measure.

As an exception, you can change the languages that a couple of the characters speak. It may be fun for you to see them try to speak your native language.  And that doesn't really affect the quality of the argument.

In [ ]:
# example
trollFace2 = characters.trollFace.replace(languages = ["Chinese", "Spanish"])
rich.print(trollFace2)
simulated_dialogue(argubots.alice, CharacterAgent(trollFace2), 6)

Character(
    name='TrollFace',
    languages=['Chinese', 'Spanish'],
    persona='a troll who loves to ridicule everyone and everything',
    conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
    conversation_starters=[
        'Do you think Donald Trump will be a good president?',
        'Do you think Joe Biden has been a good president?'
    ]
)

(Alice) Do you think Joe Biden has been a good president?
(TrollFace) 哈哈，拜登的表现真是让人忍俊不禁，简直像在看一场闹剧！
(Alice) 我理解你的看法，很多人也会对他的某些决策感到失望。不过，考虑一下，作为总统，他面临的挑战非常复杂，尤其是在疫情、经济和国际关系等方面，可能需要更长的时间来评估他的整体表现。
(TrollFace) 哦，复杂挑战？我觉得他只是给我们带来了更多的搞笑时刻，真是太有趣了！
(Alice) 我明白这样的感觉，许多选民也喜欢通过幽默来解读政治。然而，政治领导的影响往往是深远的，幽默背后，决策对国家安全、经济和民生可能有着重大影响，值得我们认真思考这些决策的后果。
(TrollFace) 认真思考？哈哈，那我宁愿看看他的决策怎么把国家搞得像一场笑话展览！

### Efficiency: Batched generation?

Notice that we are making a separate LLM call to generate each turn of the dialogue.  When we generate the $n^\text{th}$ turn, we send the server the whole dialogue history — the previous $n\!-\!1$ turns — along with some instructions.  The server has to re-encode it with the Transformer, and it charges us for doing so (see the "input token" costs in `tracking.py`).  

That is probably inevitable for real dialogue.  But for simulated dialogue, a more efficient approach would be to generate the whole dialogue between Alice and Bob in one LLM call.  Then you would be charged just once for each dialogue turn.  Under this approach, the Transformer encodes each token as soon as it is generated (see the "output token" costs in `tracking.py`).  The encoded token stays in the context throughout the dialogue, so it doesn't have to be re-encoded on a later call.  There is no later call.  

Under current pricing models, that would reduce the dollar cost of generating $n$ turns from $O(n^2)$ to $O(n)$.  

However, the pricing model doesn't quite reflect the computational costs.  
* ![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png) Using $O(\cdot)$ notation, what is the total number of floating-point operations needed to generate $n$ turns under each approach?  
* ![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png) Parallelism may help reduce the runtime.  Using $O(\cdot)$ notation, what is the total number of seconds needed to generate $n$ turns under each approach?  (Assume that the GPU is big enough, relative to $n$, that it can encode all input tokens in parallel.)

The problem with the more efficient approach is that it gives you no way to change the instructions (the system prompt) each time we switch from Alice to Bob and back again.  You'd need to generate the whole conversation using a single set of instructions.

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Can you get this to work?  Specifically, try completing the cell below.  You don't have to use the `Agent` or `Dialogue` classes.  It's okay to just throw together something like the `complete()` method above.  Just see whether you can manage to prompt gpt-4o-mini to generate a multi-turn dialogue between two characters who have different personalities and goals.  Is the quality better or worse than generating one turn at a time with different instructions?

In [26]:
# Like `simulated_dialogue` in `simulate.py`.  However, this one is called on two
# Characters, not two Agents, and it returns a string rather than a Dialogue.

from tracking import default_client, default_model
from characters import Character
def simulated_dialogue_batch(a: Character, b: Character, turns: int = 6, *, starter=True) -> str:
    # Construct a System Prompt containing both personas
    system_instructions = (
        f"Write a dialogue between {a.name} and {b.name}. "
        f"{a.name}'s persona is: {a.persona}. Style: {a.conversational_style}. "
        f"{b.name}'s persona is: {b.persona}. Style: {b.conversational_style}. "
        f"Write exactly {turns} turns of dialogue."
    )

    # determine the starting message
    # if starter is True then A starts. 
    if starter:
        first_line = a.conversation_starters[0] if a.conversation_starters else "Hello"
    else:
        first_line = b.conversation_starters[0] if b.conversatoin_starters else "Hello"
        
    assert first_line, "Logic error detected"

    # create the user prompt to trigger the generation
    user_content = f"The conversation starts with {a.name} saying: \"{first_line}\". Please continue the conversation."

    # call the OpenAI API
    response = default_client.chat.completions.create(
        model="gpt-4o-mini",  # or default_model
        messages=[
            {
                "role": "system", 
                "content": system_instructions
                },
            {
                "role": "user", 
                "content": user_content
                }
        ]
    )

    return response.choices[0].message.content 
    
    complete()
answer: str = simulated_dialogue_batch(characters.bob, characters.cara)
print(answer)

**Bob:** Do you think it's ok to eat meat? 

**Cara:** Well, I do think it’s okay, actually. It’s a personal choice, you know? 

**Bob:** I understand that, but have you considered the environmental impact and ethical concerns of meat consumption? 

**Cara:** I have, but I also believe in a balanced diet that includes meat. It gives me the energy I need, and I enjoy it. 

**Bob:** I respect your choice, but I really believe the benefits of a vegetarian lifestyle far outweigh the joy of eating meat. 

**Cara:** And I respect your views too, Bob, but please don’t push me to give up something that I enjoy.


In [27]:
simulated_dialogue(agents.bob, agents.cara)

(Bob) Do you think it's ok to eat meat?
(Cara) Yes, I believe eating meat is perfectly fine.
(Bob) I appreciate your perspective, but I encourage you to consider the benefits of a vegetarian lifestyle for both health and the environment.
(Cara) I appreciate your encouragement, but I prefer to stick with my carnivorous diet.
(Bob) That's completely your choice, and I respect your decision, though I hope you might explore vegetarian options someday.
(Cara) Thank you for respecting my choice; I prefer to enjoy my meat without exploring other options.

In [28]:
simulated_dialogue(agents.eve, agents.trollFace)

(Eve) Do you think Kamala Harris was a good vice-president?
(TrollFace) Oh sure, if you think being a VP means mastering the art of awkward pauses and cringy laughs, then she's an absolute superstar!
(Eve) That’s an interesting take! What do you think she could have done differently to improve her image?
(TrollFace) Maybe she could have tried taking a crash course in public speaking—because nothing says "leader" like fumbling over your words like a toddler on roller skates!
(Eve) That’s a valid point! Have you noticed any other politicians who seem to struggle with public speaking?
(TrollFace) Oh, absolutely! It’s like a talent show for the awkward—just pick any politician and you’ll find someone who could make a mime look like a charismatic genius!

# Model-based evaluation

What is our goal for the argubot?  We'd like it to broaden the thinking of the (simulated) human that it is talking to.  Indeed, that's what Alice's prompt tells Alice to do.

This goal is inspired by the recent paper [Opening up Minds with Argumentative Dialogues](https://aclanthology.org/2022.findings-emnlp.335/), which collected human-human dialogues:

> In this work, we focus on argumentative dialogues that aim to open up (rather than change) people’s minds to help them become more understanding to views that are unfamiliar or in opposition to their own convictions. ... Success of the dialogue is measured as the change in the participant’s stance towards those who hold opinions different to theirs.

Arguments of this sort are not like chess or tennis games, with an actual winner.  The argubot will almost never hear a human say "You have convinced me that I was wrong."  But the argubot did a good job if the human developed **increased understanding and respect for an opposing point of view**.  

To find out whether this happened, we can use a questionnaire to ask the human what they thought after the dialogue.  For example, after Alice talks to Bob, we'll ask Bob to evaluate what he thinks of Alice's views.  Of course, that depends on his personality — Alice needs to talk to him in a way that reaches *him* (as much as possible).  We'll also ask an outside observer to evaluate whether Alice handled the conversation with Bob well.

Of course, we're still not going to use real humans.  Bob is a fake person, and so is the outside observer (whose name is Judge Wise).
Using an LLM as an eval metric is known as *model-based evaluation*.  It has pros and cons:
* It is cheaper, faster, and more replicable than hiring actual humans to do the evaluation.  
* It might give different answers than what humans would give.   

Social scientists usually refer to a metric's **reliability** (low variance) and **validity** (low bias).  So the points above say that model-based evaluation is reliable but not necessarily valid.  In general, an LLM-based metric (like any metric) needs to be validated to confirm that it really does measure what it claims to measure.  (For example, that it correlates strongly with some other measure that we already trust.)  In this homework, we'll skip this step and just pray that the metric is reasonable.

To see how this works out in practice, open up the `demo` notebook, which walks you through the evaluation protocol.  You'll see how to call the [starter code](http://cs.jhu.edu/~jason/465/hw/llm), how it talks to the LLM behind the scenes, and what it is able to accomplish. 

To help to validate the metric, check that Airhead gets a low score.  (It should!)

# Reading the starter code

The `demo` notebook gave you a good high-level picture of what the starter code is doing.  So now you're probably curious about the details.  Now that you've had the view from the top, here's a good bottom-up order in which to study the code.  You don't need to understand every detail, but you will need to understand enough to call it and extend it.

* `character.py`.  The `Character` class is short and easy.

* `dialogue.py`.  The `Dialogue` class is meant to serve as a record of a natural-language conversation among any number of humans and/or agents.  On each *turn* of the dialogue, one of the speakers says something.  

   The dialogue's sequence of turns may remind you of the sequence of messages that is sent to OpenAI's chat completions API.  But the OpenAI messages are only labeled with the 4 special roles `user`, `assistant`, `tool`, and `system`.  Those are not quite the same thing as human speakers.  And the OpenAI messages do not necessarily form a natural-language dialogue: some of the messages are dealing with instructions, few-shot prompting, tool use, and so on.  The `agents.dialogue_to_openai` function in the next module will map a `Dialogue` to a (hopefully appropriate) sequence of messages for asking the LLM to extend that dialogue.

* `agents.py`.  This module sets up the problem of automatically predicting the next turn in a dialogue, by implementing an `Agent`'s `response()` method.  The `Agent` base class also has some simple convenience methods that you should look at.  

   Some important subclasses of `Agent` are defined here as well.  However, you may want to skip over `EvaluationAgent` and come back to it only when you read `evaluate.py`.

* `simulate.py` makes agents talk to one another, which we'll do during evaluation.

* `argubots.py` starts to describe some useful agents.  One of them makes use of the `kialo.py` module, which gives access to a database of arguments.

* `evaluate.py` makes use of `simulate.simulated_dialogue` to `agents.EvaluationAgent` to evaluate an argubot.

* We also have a couple of utility modules.  These aren't about NLP; look inside if needed.  `logging_cm.py` is what enabled the context manager `with LoggingContext(...):` in the demo notebook.  `tracking.py` sets some global defaults about how to use the OpenAI API, and arranges to track how many tokens we're paying for when you call it.







# Similarity-based retrieval: Looking up relevant responses

Now, it is fine to prompt an LLM to generate text, but there are other methods!
There is a long history of machine learning methods that "memorize" the training data.
To make a prediction or decision at test time, they consult the stored training examples
that are most similar to the training situation.

_Similarity-based retrieval_ means that given a document $x$, you find the "most similar" documents $y \in Y$, where $Y$ is a given collection of documents.  The most common way to do this is to maximize the _cosine similarity_ $\vec{e}(x) \cdot \vec{e}(y)$, where $\vec{e}(\cdot)$ is an embedding function.

Should we use the OpenAI embedding model?  We could, but we would have to precompute $\vec{e}(y)$ for all $y \in Y$, and store all these vectors in a data structure that supports some type of fast similarity-based search (e.g., using the [FAISS](https://faiss.ai/index.html) package).  An alternative would be to upload the documents to OpenAI and let OpenAI compute and store the embeddings.  We would then use their similarity-based [retrieval tool](https://platform.openai.com/docs/assistants/overview).

A simpler and faster approach—which sometimes even works better—is to use a _bag of tokens_ embedding function: Define $\vec{e}(y)$ to be the vector in $\mathbb{R}^V$ that records the count of each type of token in a tokenized version of $y$, where $V$ is the token vocabulary.  [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is a refined variant of that idea, where the counts are adjusted in 3 ways: 

* smooth the counts
* normalize for the document length $|y|$ so that longer documents $y$ are not more likely to be retrieved
* downweight tokens that are more common in the corpus (such as ` the` or `ing`) since they provide less information about the content of the document


You might like to play with the `rank_bm25` package ([documentation](https://pypi.org/project/rank-bm25/)).  It is widely used and very easy to use.

In [29]:
from rank_bm25 import BM25Okapi as BM25_Index   # the standard BM25 method
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")  # how this model will tokenize

corpus = [
    "Marry had a little lamb, little lamb, little lamb.",
    "I love reading books about history and science",
    "The weather today is sunny and bright",
    "Natural Language Processing is fascinating and Hard"
]

toks = [tokenizer.encode(cor) for cor in corpus]
bm25 = BM25_Index(toks)

query = "Natural Language Processing is bright but little hard"

scores = bm25.get_scores(tokenizer.encode(query))
print(scores)



# experiment here!  You could try the examples in the rank_bm25 documentation.

[1.25925372 0.         0.93109655 2.79328965]


## The Kialo corpus

How can we use similarity-based retrieval to help build an argubot?  It's largely about having the right data!

[Kialo](kialo.com) is a collaboratively edited website (like Wikipedia) for discussing political and philosophical topics.  For each topic, the contributors construct a tree of _claims_.  Each claim is a natural-language sentence (usually), and each of its children is another claim that supports it ("pro") or opposes it ("con").  For example, check out the tree rooted at the claim ["All humans should be vegan."](https://www.kialo.com/all-humans-should-be-vegan-2762).

We provide a class `Kialo` for browsing a collection of such trees.  Please read the [source code](https://www.cs.jhu.edu/~jason/465/hw-llm) in `kialo.py`.  The class constructor reads in text files that are [exported Kialo discussions](https://support.kialo.com/en/hc/exporting-a-discussion/); we have provided some in the [data directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data).  The class includes a BM25 index, to be able to find claims that are relevant to a given string.

In [30]:
from kialo import Kialo

Ok, let's pull the retrieved discussions (the `.txt` files) into our data structure.

For BM25 purposes, we have to be able to turn each document (that is, each Kialo claim) as a list of string or integer tokens. 

In [31]:
from typing import List
import glob

# kialo = Kialo(glob.glob("data/*"), tokenizer=tokenizer.encode)  # using the LLM's tokenizer doesn't work here for some reason
kialo = Kialo(glob.glob("data/*"))  # use simple default tokenizer
f"This Kialo subset contains {len(kialo)} claims"

'This Kialo subset contains 6251 claims'

Let's use sampling to see what kind of stuff is in the data structure.

In [32]:
kialo.random_chain()   # just a single random claim

['Utilitarian ethics is not only a disputed way of thinking, but is very difficult to implement (for instance, what is the appropriate time span of pain/pleasure that we are computing?), especially in nature, which does not care about utilitarianism.']

In [33]:
kialo.random_chain(n=4)

['Humans should stop eating animal meat.',
 'The taste of meat is delicious and brings many people pleasure in a manner that vegetarian food cannot fully imitate.',
 'It is because the taste of meat is so good and popular that scientists try to imitate it and create vegetarian "meat" products.',
 'They do not imitate meat because of the taste, but because consumers do not want to miss out on certain dishes that usually contain meat, for example burgers.']

### Similarity-based retrieval from the Kialo corpus

Let's try it, using BM25!

In [34]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

We can restrict to claims for which the Kialo data structure has at least one counterargument ("con" child).

In [35]:
kialo.closest_claims("animal populations", n=10, kind='has_cons')

['Industrial agriculture can dangerously decrease animal populations.',
 'Effective vegan methods to control animal populations exist.',
 'Human-introduced species have historically devastated local wildlife populations across the world.',
 'COVID-19 has devastated prison populations, whose lives are the responsibility of the state.',
 'High demand for vegan foods may hike prices for local populations that previously depended on them.',
 'It is generally poorer countries that have expanding populations. The first world has now reached a point of stagnant population growth - even declining populations, as in the case of Japan and others. The inability of poorer countries to control their populations should not impact the lives of those in the first world. The first world having earned their luxuries and should not be denied them.',
 'Vegan populations are, on average, less likely to suffer from obesity, a major risk factor for many diseases and health problems.',
 'Humans, as apex preda

In [36]:
c = _[0]    # first claim above
print("Parent claim:\n\t" + str(kialo.parents[c]))
print("Claim:\n\t" + c)
print('\n\t* '.join(["Pro children:"] + kialo.pros[c]))
print('\n\t* '.join(["Con children:"] + kialo.cons[c]))

Parent claim:
	In a vegan world, fewer species would be at risk of extinction.
Claim:
	Industrial agriculture can dangerously decrease animal populations.
Pro children:
	* The fishing industry is especially deleterious to the ocean's biota due to overfishing and the disruption of the natural ecosystem.
	* Up to 100,000 species go extinct annually, largely due to the environmental effects of animal agriculture.
Con children:
	* Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.


### Does BM25 really work?

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Unfortunately, we see that `"animal population"` gives quite different results from `"animal populations"`.  Why is that and how would you fix it?  

Also, both queries seem to retrieve some claims that are talking about human populations, not animal populations.  Why is that and how would you fix it?

In [63]:
kialo.closest_claims("animal population",10)

['As long as our ability to produce both animal feed crops and food crops for our human population are not exceeded, this point is irrelevant.',
 "36% of the calories produced by the world's crops are being used for animal feed, of which only 12% then turn into animal products that can be eaten by the human population. That is a waste of 24% of the world's crops.",
 'The claim that "most of the cultural shift and loss is due to mostly vegan cultures turning to animal products" is completely unfounded, and the Brokpa people which you cited are an outlier as a group that has a population of less than 70k people. Worldwide the population of vegan people has only increased.',
 "Developed nations are fueling the 3rd world and underdeveloped nation's population boom by exporting/donating food to areas that cannot sustain their current population.",
 'This argument assumes that sentience is the only objection to the consumption of animal products, failing to address the issues involved with t

## A retrieval bot (Akiko)

The starter code defines a simple argubot named Akiko (defined in `argubots.py`) that doesn't use an LLM at all.  It simply finds a Kialo claim that is similar to what the human just said, and responds with one of the Kialo counterarguments to that claim.

You already watched Akiko argue with Darius in `demo.py`.  If you look at the log messages, you'll see the claims that Akiko retrieved, as well as the LLM calls that Darius made.  

You can talk to Akiko yourself now.  (Remember that Akiko only knows about subjects that it read about in the [`data` directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data/).  If you want to talk about something else, you can add more conversations from [kialo.com]; see the [LICENSE](https://www.cs.jhu.edu/~jason/465/hw-llm/data/LICENSE) file.)


In [64]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiko.converse()

## Making your own retrieval bot (Akiki)

As you can see when talking to Akiko yourself, Akiko does poorly when responding to a short or vague dialogue turn (like "Yes"), because the "closest claim" in Kialo may be about a totally different subject.  Akiko does much better at responding to a long and specific statement.  

So try implementing a new argubot, called Akiki, that is very much like Akiko but does a better job of staying on topic in such cases.  It should be able to **look at more of the dialogue** than the most recent turn.  But the most recent dialogue turn should still be "more important" than earlier turns.  

The details are up to you.  Here are a few things you could try:
* include earlier dialogue turns in the BM25 query only if the BM25 similarity is too low without them
* weight more recent turns more heavily in the BM25 query (how can you arrange that?)
* treat the human's earlier turns differently from Akiki's own previous turns

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Implement your new bot Akiki in `argubots.py`, and adjust it until `argubots.akiki.converse()` seems to do a better job of answering your short turns, compared to `argubots.akiko.converse()`.  Make sure it still gives appropriate reponses to long turns, too.  Give some examples in the notebook of what worked well and badly, with discussion.

### Evaluating Akiki

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Finally, do a more formal evaluation to verify whether Akiki really does better than Akiko on this dimension.  This is a way to check that you're not just fooling yourself.  

1. Make a new `Agent` called "Shorty" that often (but not always) gives short responses.  
    * Shorty's conversation starters should be on topics that Kialo knows about.  
    * Shorty could be a pure `LLMAgent` such as a `CharacterAgent` with a particular `conversational_style`.  Or it could use a mixed strategy of calling the LLM on some turns and not others.
2. Generate several *Akiko*-Shorty dialogues and several *Akiki*-Shorty dialogues, using `simulated_dialogue`.
3. Evaluate each of those dialogues by asking Judge Wise **how well the argubot stayed on topic**.  You should write this prompt carefully so that Judge Wise gives meaningful scores.  (Before you do this evaluation step, adjust the prompt until it seems to work well on a small subset of the dialogues, Otherwise Judge Wise won't be so wise!)  
4. Compare Akiko and Akiki's mean scores on this new evaluation criterion (which you can call `'focused'`). Ideally, compute a 95% confidence interval on the difference of means, using [this calculator](https://www.statskingdom.com/difference-confidence-interval-calculator.html).  If you don't get statistical significance, then your evaluation set wasn't large enough, so go back to step 2 and run the comparison again (from scratch) by generating a larger set of dialogues with Shorty for each argubot.

You can do all those steps in the notebook, writing _ad hoc_ code.  You don't have to write general-purpose methods or classes.

## Retrieval-augmented generation (Aragorn)

The real weaknesses of Akiko and Akiki:
* They can only make statements that are already in Kialo.  
* They don't respond to the user's actual statement, but to a single retrieved Kialo claim that may not accurately reflect the user's position (it just overlaps in words).

But we also have access to an LLM, which is able to generate new, contextually appropriate text (as Alice does).

In this section, you will create an argubot named [Aragorn](https://tolkiengateway.net/wiki/Riddle_of_Strider), who is basically the love child of Akiki and Alice, combining the high-quality specific content of Kialo with the broad competence of an LLM.  

The RAG in aRAGorn's name stands for **retrieval-augmented generation**.  Aragorn is an agent that will take 3 steps to compute its `Agent.response()`:

1. **Query formation step**: Ask the LLM what claim should be responded to.  For
   example, consider the following dialogue:
    > ...
    > Aragorn: Fortunately, the vaccine was developed in record time.
    > Human: Sounds fishy.

    "Sounds fishy" is exactly the kind of statement that Akiko had trouble using
    as a Kialo query.  But Aragorn shows the *whole dialogue* to the LLM, and
    asks the LLM what the human's *last turn* was really saying or implying, in
    that context. The LLM answers with a much longer statement:

    > Human [paraphrased]: A vaccine that was developed very quickly cannot be trusted.
    > If its developers are claiming that it is safe and effective, I question their motives.

    This paraphrase makes an explicit claim and can be better understood without the context.
    It also contains many more word types, which makes it more likely that BM25 will be able
    to find a Kialo claim with a nontrivial number of those types. 

2. **Retrieval step**: Look up claims in Kialo that are similar to the explicit
   claim.  Create a short "document" that describes some of those claims and
   their neighbors on Kialo.

3. **Retrieval-augmented generation**: Prompt the LLM to generate the response
   (like any `LLMAgent`).  But include the new "document" somewhere in the LLM
   prompt, in a way that it influences the response. 
   
   Thus, the LLM can respond in a way that is appropriate to the dialogue but
   also draws on the curated information that was retrieved in Kialo.  After
   all, it is a Transformer and can attend to both!

Here's an example of the kind of document you might create at the retrieval step, though it may be possible
to do better than this:

In [65]:
# refers to global `kialo` as defined above
def kialo_responses(s: str) -> str:
    c = kialo.closest_claims(s, kind='has_cons')[0]
    result = f'One possibly related claim from the Kialo debate website:\n\t"{c}"'
    if kialo.pros[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users in favor of that claim:"] + kialo.pros[c])
    if kialo.cons[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users against that claim:"] + kialo.cons[c])
    return result
        
print(kialo_responses("Animal flesh is yucky to think about, yet delicious."))

One possibly related claim from the Kialo debate website:
	"So many people are worried about animals but don't even think twice when walking by a homeless person on the streets. It's preposterous. How about we worry about our own kind first and then start talking about animals."
Some arguments from other Kialo users against that claim:
	* This implies that caring for animals or caring for people is a binary choice. It isn't. There are those who are well placed and willing to care for people and those who prefer to serve the animal kingdom. As a species we don't just have one idea at a time and follow that to conclusion before we pursue another. It benefits all if humans divide their attentions between various issues and problems we face.
	* Humans have freedom of choice to some extent, animals subdued by humans don't. The very intention of help urges it to go where is most needed. And so far never was any biggest, flagrant and needless cruelty and slaughter as that towards industrial f

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
**You should implement Aragorn in `argubots.py`, just as you did for Akiki.**  Probably as an instance `aragorn` of a new class `RAGAgent` that is a subclass of `Agent` or `LLMAgent`.

### Evaluating Aragorn

![image](https://cs.jhu.edu/~jason/465/hw-llm/handin.png)
Compare Alice, Akiki, and Aragorn in the notebook, using the evaluation scheme and devset that were illustrated in `demo.ipynb`.  In other words, use `evaluate.eval_on_characters`.

Who does best?  What are the differences in the subscores and comments?  Does it matter which character you're evaluating on — maybe the different characters expoes the bots' various strenghts and weaknesses?

Try to figure out how to improve Aragorn's score.  Can you beat Alice?

Also, try evaluating them in the same way that you evaluated Akiki.  In other words, have them talk to Shorty and ask Judge Wise whether they were able to stay on topic.  This is where Aragorn should really shine, thanks to its ability to paraphrase Shorty's short utterances.



In [72]:

# the list of bots to evaluate
bots_to_evaluate = [
    argubots.alice,
    argubots.akiko,
    argubots.aragon,
]

# the set of characters to test against
devset = characters.devset

# a dictionary to store the evaluation results
evaluation_summaries = {}

print("--- Starting Evaluation ---")

with LoggingContext("evaluate"):
    for bot in bots_to_evaluate:
        print(f"Evaluating {bot.name}...")
        evaluation_summaries[bot.name] = evaluate.eval_on_characters(
            argubot=bot,
            chars=devset,  # The standard development set of characters
            reps=1,        # How many times to test against each character
            turns=3        # The number of turns in each dialogue
        )

# Now, let's print out the results in a nicely formatted way.
print("\n\n--- Evaluation Summary ---")

for bot_name, summary in evaluation_summaries.items():
    print(f"\n--- {bot_name} ---")
    rich.print(summary)
    print("\nScores:")
    rich.print(summary.mean())
    print("\nStandard Deviations:")
    rich.print(summary.sd())

print("\nEvaluation complete.")


--- Starting Evaluation ---
Evaluating Alice...


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [01:12<00:00, 14.43s/it]


You just spent $0.00 of NLP money to evaluate <LLMAgent Alice>                                      ]8;id=146920;file:///home/kahl/codes/ai/nlp/hw_8/nlp-hw8/evaluate.py\evaluate.py]8;;\:]8;id=28013;file:///home/kahl/codes/ai/nlp/hw_8/nlp-hw8/evaluate.py#296\296]8;;\

Evaluating Akiko...


...


Scores:


{'engaged': 2.8, 'informed': 3.4, 'intelligent': 3.4, 'moral': 3.2, 'skilled': 7.2, 'TOTAL': 20.0}


Standard Deviations:


{
    'engaged': 0.8366600265340751,
    'informed': 0.5477225575051667,
    'intelligent': 0.5477225575051667,
    'moral': 0.44721359549995715,
    'skilled': 0.4472135954999611,
    'TOTAL': 2.345207879911715
}


Evaluation complete.


# Awsom

![image](handin.png)
Add another LLM-based argubot to `argubots.py`.  
Call it Awsom.  Try to make it get the best score, according to `evaluate.eval_on_characters`.
Explain what you did and discuss what you found.

(This corresponds to the `--awesome` flag on earlier assignments, but naming the character "Awesome" might bias the evaluation system, so we changed the spelling!)

If the idea was interesting and you implemented it correctly and well, it's okay if it turns out not to help the score.  Many good ideas don't work.  That's why you need to keep finding and trying new good ideas.  (Sometimes they do help, but in a way that is not picked up by the scoring metric.)

You may want to use Aragorn or Alice as your starting point.
Then see if you can find tricks that will get a more awesome score for Awsom.
How you choose to do that is up to you, but some ideas are below.

(Reminder: **Don't change evaluation.**  Just build a better argubot.)

In [ ]:
bot_to_evaluate = argubots.awsome

# the set of characters to test against
devset = characters.devset

# a dictionary to store the evaluation results
evaluation_summary = {}

print(f"--- Starting Evaluation for {bot_to_evaluate.name} ---")

# run the evaluation
evaluation_summary = evaluate.eval_on_characters(
    argubot=bot_to_evaluate,
    chars=devset,  # The standard development set of characters
    reps=1,        # How many times to test against each character
    turns=3        # The number of turns in each dialogue
)

print(f"\n\n--- Evaluation Summary for {bot_to_evaluate.name} ---")

rich.print(evaluation_summary)
print("\nScores:")
rich.print(evaluation_summary.mean())
print("\nStandard Deviations:")
rich.print(evaluation_summary.sd())

print("\nEvaluation complete.")


--- Starting Evaluation for Awsome ---


100%|██████████| 5/5 [01:28<00:00, 17.77s/it]


You just spent $0.00 of NLP money to evaluate <LLMAgent Awsome>                                     ]8;id=38810;file:///home/kahl/codes/ai/nlp/hw_8/nlp-hw8/evaluate.py\evaluate.py]8;;\:]8;id=953432;file:///home/kahl/codes/ai/nlp/hw_8/nlp-hw8/evaluate.py#296\296]8;;\



--- Evaluation Summary for Awsome ---


<Eval of 5 dialogues: {'engaged': 4.0, 'informed': 3.2, 'intelligent': 3.4, 'moral': 3.2, 'skilled': 7.4, 'TOTAL': 
21.2}>
Standard deviations: {'engaged': 0.0, 'informed': 0.44721359549995715, 'intelligent': 0.5477225575051667, 'moral': 
0.44721359549995715, 'skilled': 0.5477225575051635, 'TOTAL': 1.7888543819998444}

Comments from participant_overview question:
(Bob) Awsome didn't outright disagree with me but acknowledged the nutritional benefits of including some meat in 
diets, which suggests a difference in perspective regarding the necessity of meat for health. The conversation was 
respectful and open, allowing for a constructive exchange of ideas.

Awsome could have done better by asking more about specific vegetarian sources for essential nutrients or 
expressing curiosity about how to address potential nutritional gaps in a vegetarian diet. This would have 
furthered the discussion and allowed for a deeper exploration of the vegetarian lifestyle.
(Cara) Awsome didn't directly disagree with me; rather, they acknowledged my perspective while introducing a new 
topic about the environmental impact of meat consumption. The conversation was respectful and open, which is a good
approach. However, Awsome could have done better by not assuming that I hadn't considered the environmental aspects
of meat consumption. It would have been more effective to ask about my views on that specific topic instead of 
implying that it’s often overlooked.
(Darius) Awsome disagreed with me primarily on the issue of mandatory COVID vaccinations, emphasizing the 
importance of individual freedom and personal choice. While he acknowledged the benefits of vaccination, he raised 
concerns about respecting individual rights and beliefs.

In my opinion, the conversation was constructive, as it allowed for a discussion of both public health benefits and
individual rights. However, Awsome could have strengthened his argument by providing specific examples or data that
illustrate the impact of personal beliefs on vaccination rates or public health outcomes. This would have made the 
conversation more fact-based and grounded in evidence, which is essential for a robust discussion on such a 
critical topic.
(Eve) Awsome didn't explicitly disagree with me; rather, they provided a balanced view on the topic of mandatory 
vaccinations, acknowledging both the public health benefits and the concerns about individual autonomy. The 
conversation flowed well, with Awsome sharing their thoughts and inviting me to share mine, which is a good way to 
engage in a discussion.

However, Awsome could have been more direct in expressing their own stance on mandatory vaccinations instead of 
just outlining the pros and cons. This would have made the conversation more personal and engaging. Overall, it was
a thoughtful exchange!
(TrollFace) Awsome didn't actually disagree with me; they seemed to enjoy my bakery analogy and wanted to dive 
deeper into my thoughts on J.D. Vance. The conversation was light-hearted and playful, which is always a win in my 
book!

As for where Awsome could have done better, they could have challenged my opinion a bit more instead of just 
agreeing. A little friendly debate would have spiced things up! But hey, who am I to complain? I love a good laugh!

Comments from observer_mindopening question:
(Judge Wise) Awsome introduced the perspective that while a vegetarian lifestyle has health and environmental 
benefits, there are also nutritional advantages to including some meat in the diet. This acknowledgment of the 
complexity of the issue helps to create a more balanced conversation. Awsome's approach encourages Bob to consider 
alternative viewpoints and fosters a constructive dialogue by expressing appreciation for Bob's perspective while 
also presenting counterarguments.

Overall, this attempt was successful in promoting a constructive conversation. Awsome's respectful tone and 
willingness to engage with Bob's views likely made it easi


Scores:


{'engaged': 4.0, 'informed': 3.2, 'intelligent': 3.4, 'moral': 3.2, 'skilled': 7.4, 'TOTAL': 21.2}


Standard Deviations:


{
    'engaged': 0.0,
    'informed': 0.44721359549995715,
    'intelligent': 0.5477225575051667,
    'moral': 0.44721359549995715,
    'skilled': 0.5477225575051635,
    'TOTAL': 1.7888543819998444
}


Evaluation complete.


## [Possible strategy] Prompt engineering

A good first thing to do is to experiment with Alice's prompt.  
The wording and level of detail in the prompt can be quite important.
Often, NLP engineers will change their prompt to try to address 
problems that they've seen in the responses.

Because it's "just" text editing, this won't get full credit by itself unless you make a real discovery.
But it requires intelligence, care, experimentation, and alertness to the language of the responses and the
language of the prompts.  And you'll develop some intuitions about what helps and what doesn't.
It is certainly worthwhile.

Of course, people have tried to develop methods to search for good prompts automatically, or semi-automatically with human guidance.  
So you could additionally try out SAMMO or DSPy -- both have multiple tutorials and are downloadable from github.

If you try this, what worked well for you?

## [Possible strategy] Chain of thought / Planning

The evaluation functions in `evaluate.py` asked each `EvaluationAgent` a "warmup question" before continuing with the real question.  That is an example of chain-of-thought (CoT) reasoning, where the LLM is encouraged to talk through the problem for a few sentences before giving the answer.  CoT sometimes improves performance.

Instead of using one prompt, could you help an `LLMAgent` argubot (like Alice) do better by having think aloud before it gives an answer?  For example, each time the human speaks, your argubot (Awsom) could prompt the LLM to think about the human's ideas/motivations/personality, and to come up with a plan for how to open the human's mind. 

For example, you might structure this as a `Dialogue` among three participants, like this:
> Awsom (to Eve): Do you think COVID vaccines should be mandatory?
>
> Eve: Have you ever gotten vaccinated yourself?<br>
>
> Awsom (private thought): I don't know Eve's opinions yet, so I can't push back.  Eve might be avoiding my question because she doesn't want to get into a political argument.  So let's see if we can get her to express an opinion on something less political.  Maybe something more personal ... like whether vaccines are scary.
>
> Awsom (to Eve): In fact I have, and so have millions of others. But some people seem scared about getting the vaccine.  

One way to trigger this kind of analysis is to present a `Dialogue.script()` to Awsom (or to an observer), and ask an open-ended question about it.  Or you could ask a series of more specific questions.  That is basically what `eval_by_participant` and `eval_by_observer` do.  But here the argubot itself is doing it, rather than the evaluation framework.

Eve would be shown only the turns that are spoken aloud.  However, when analyzing and responding, Awsom would get to see Awsom's own private thoughts as well.


## [Possible strategy] Dense embeddings

BM25 uses sparse embeddings — a document's embedding vector is mostly zeroes, since the non-zero coordinates correspond to the specific words (tokens) that appear in the document.

But perhaps dense embeddings of documents would improve Aragorn by reading the text and abstracting away from the words, in a way that actually cares about word order.  So, try it!

How?  As mentioned earlier in this notebook, you could compute the embeddings yourself and put them in a FAISS index. Or you could figure out how to use OpenAI's [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) API.

## [Possible strategy] Few-shot prompting

 In this homework, often an agent prompted a language model only with instructions.  Can you find a place where giving a few _examples_ would also improve performance?  You will have to write the examples, and you will have to add them to the sequence of messages that your agent sends to the OpenAI API.  See the sentence-reversal illustration earlier in this notebook.

One good opportunity is in the query formation step of RAG.  This is a tricky task.  The LLM is supposed to state the user's implicit claim in a form that looks like a Kialo claim (or, more precisely, a form that will work well as a Kialo query).  It probably doesn't know what Kialo claims look like.  So you could show it by way of example.  This would also show it what you mean by the user's "implicit claim."


## [Possible strategy] Using tools in the approved way

Aragorn's step 1 (query formation) is basically getting the LLM to generate a function call like
```
kialo_thoughts("A vaccine that was developed very quickly ...")
```
which Aragorn will execute at step 2 (retrieval), sending the results back to the LLM as part of step 3.

In this context, `kialo_thoughts` is an example of a **tool** (that is, a function) that the
LLM can or must use before it gives its response.

The tool is _not_ something that runs on the LLM server.  It is written by you
in Python and executed by you.  The function call above, including the text `"A
vaccine that was ..."`, is the part that is generated by the LLM.

The OpenAI API has [special support](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models) for calling the LLM in a way that will _allow_ it to generate a tool call ([tools](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools)) or _force_ it to do so ([tool_choice](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tool_choice)).  You can then send the tool's result back to the LLM [as part of your message sequence](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages).

So, you could modify Aragorn to use tools properly.  Maybe that will help, simply because the LLM was trained on message sequences that included tool use.  It should know to pay attention to the tool portions of the prompt when they are relevant, and ignore them when they are not.

The `client.chat.completions.create()` method would need to be told about the tool by using the `tools` keyword argument, with a value something the one below.

If `d` is a `Dialogue`, you should be able to call `d.response()` with the `tools` keyword argument.  This will be passed on to `client.chat.completions.create()` as desired.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "kialo_thoughts",
            "description": "Given a claim by the user, find a similar claim on the Kialo website and return its pro and con responses",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_topic": {
                        "type": "string",
                        "description": "A claim that was made explicitly or implicitly by the user.",
                    },
                },
                "required": ["search_topic"],
            },
        }
    }]

## [Possible strategy] Parallel generation

The chat completions interface allows you to sample $n$ continuations of the prompt in parallel, as we saw with "the apples, bananas, cherries ..." example.  This is efficient because it requires only 1 request to the LLM server and not $n$.  The latency does not scale with $n$.  Nor does the input token cost, since the prompt only has to be encoded once.

Perhaps you can find a way to make use of this?  For example, the query formulation step of RAG could generate $n$ implicit claims instead of just one.  We could then look for claims in the Kialo database that are close to _any_ of those implicit claims.

Another thing to do with multiple completions is to select among them or combine them.  For example, suppose we prompt the LLM to generate completions of the form $(s,t,r)$ where $s$ is an answer, $t$ evaluates that answer, and $r$ is a numerical score or reward based on that evaluation.  ("Write a poem, then tell us about its rhyme and rhythm problems, then give your score.")  
* If we sample multiple completions $(s_1,t_1,r_1), \ldots, (s_n,t_n,r_n)$ in parallel, then we can return the $s_i$ whose $r_i$ is largest.  
* Or if we sample $s$ and then multiple continuations $(t_1,r_1), \ldots, (t_n,r_n)$, then we can return the mean score $\sum_i r_i/n$ as a reduced-variance score for $s$, which averages over diverse textual evaluations that might consider different aspects of $s$.

Note that when you call the chat completions interface with $n > 1$, you specfy just 1 input prompt and get $n$ different output completions.  Since the input prompt must be the same for all outputs, it is necessary to sample all of $(s,t,r)$ or all of $(t,r)$ with a single call to the LLM.

Alternatively, it is possible to reduce latency by submitting multiple requests to the server in parallel (see "async usage" [here](https://pypi.org/project/openai/)).  In this case the input prompts can be different, although you now have to pay to encode all of them separately.  This facility could speed up evaluation without changing its results; that's a worthwhile thing to try for extra credit!


# [Extra credit] Adversarial testing (Anansi)

![image](handinec.png)
Finally, let's test whether our eval metric `evaluate.eval_on_characters` is vulnerable to adversarial gaming.  Remember [Goodhart's Law](https://en.wikipedia.org/wiki/Goodhart%27s_law) ...

Add one more argubot to `argubots.py`.
Call it [Anansi](https://www.britannica.com/topic/Ananse), after the trickster character from folklore.

Can you make Anansi *fool* the judges into giving him a high score?  (Higher than some of the earlier argubots, while actually being worse at the task?)  **Any sneaky way of constructing Anansi's responses is fair game.**  The goal is to do well under automated evaluation on a held-out test set.  That is, Anansi should continue to score highly when talking to a character who is not in `evaluate.dev_chars` = {Bob, Cara, Darius, Eve, TrollFace}, when judged both by the character he is talking to and by Judge Wise.

To do well at this, figure out what the judges "want" -- what they might reward or respond positively to -- and how to give it to them.  This might be done by pure prompt engineering, or with additional computation (perhaps making use of additional LLM calls or other resources).  Again, explain what you did, and discuss how it worked out on the dev set.  Feel free to mention other ideas you had, too.